In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        os.path.join(dirname, filename)

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import tensorflow as tf
from tensorflow import keras

from keras.layers import Input, Dense, BatchNormalization, Flatten, Concatenate, Reshape, Embedding, Multiply
from keras.layers import Conv2D, Conv2DTranspose, LeakyReLU, Activation, multiply, Dropout
from keras.initializers import RandomNormal
from keras.optimizers import Adam
from keras.models import Model, Sequential
from keras.preprocessing.image import load_img

from keras.preprocessing.image import ImageDataGenerator

import matplotlib.pyplot as plt
import numpy as np

In [ ]:
from tqdm import tqdm
def load_images(path):
    source_imgs=[]
    target_imgs=[]
    
    imgs=os.listdir(path)
    for i in tqdm(range(len(imgs))):
        arr=np.array(load_img(f'{path}/{imgs[i]}', target_size=(256,512)))
        source_imgs.append(arr[:,:256,:])
        target_imgs.append(arr[:,256:,:])
        
    return np.array(source_imgs), np.array(target_imgs)

train_s, train_t=load_images('/kaggle/input/pix2pix-maps/train')
val_s, val_t=load_images('/kaggle/input/pix2pix-maps/val')


In [ ]:

for i in range(3):
    plt.subplot(2,3,i+1)
    plt.imshow(train_s[i])
    
for i in range(3,6):
    plt.subplot(2,3 ,i+1)
    plt.imshow(train_t[i-3])
    
plt.show()
    

# The discriminator is a deep convolutional neural network that performs image classification. Specifically, conditional-image classification. It takes both the source image (e.g. satellite photo) and the target image (e.g. Google maps image) as input and predicts the likelihood of whether target image is real or a fake translation of the source image.

In [ ]:
def build_discriminator(image_shape):
    
    init = RandomNormal(stddev=0.02, seed=90)
    
    inp_s=Input(shape=image_shape)
    inp_t=Input(shape=image_shape)   
    merged=Concatenate()([inp_s, inp_t])
    
    d=Conv2D(64, (4,4), strides=(2,2),  padding='same', kernel_initializer=init)(merged)
    d=LeakyReLU(alpha=0.2)(d)
    
    d=Conv2D(128, (4,4), strides=(2,2),  padding='same', kernel_initializer=init)(d) 
    d=BatchNormalization()(d)
    d=LeakyReLU(alpha=0.2)(d)
    
    d=Conv2D(256, (4,4), strides=(2,2),  padding='same', kernel_initializer=init)(d)  
    d=BatchNormalization()(d)
    d=LeakyReLU(alpha=0.2)(d)
    
    d=Conv2D(512, (4,4), strides=(2,2),  padding='same', kernel_initializer=init)(d)  
    d=BatchNormalization()(d)
    d=LeakyReLU(alpha=0.2)(d)
    
    d=Conv2D(512, (4,4), padding='same', kernel_initializer=init)(d)  
    d=BatchNormalization()(d)
    d=LeakyReLU(alpha=0.2)(d)
    
    d=Conv2D(1, (4,4), padding='same', kernel_initializer=init)(d)
    patch_out=Activation('sigmoid')(d)
    
    model=Model([inp_s, inp_t], patch_out)
    
    opt = Adam(lr=0.0002, beta_1=0.5)
    model.compile(loss='binary_crossentropy', optimizer=opt, loss_weights=[0.5])
    
    return model

image_shape=(256,256,3)
disc=build_discriminator(image_shape)
disc.summary()
    
    
    

In [ ]:
#plt.imshow(disc.predict([np.expand_dims(train_s[0], axis=0), np.expand_dims(train_t[0], axis=0)])[0])

### The generator is an encoder-decoder model using a U-Net architecture. The model takes a source image (e.g. satellite photo) and generates a target image (e.g. Google maps image). It does this by first downsampling or encoding the input image down to a bottleneck layer, then upsampling or decoding the bottleneck representation to the size of the output image. The U-Net architecture means that skip-connections are added between the encoding layers and the corresponding decoding layers, forming a U-shape.
### The encoder and decoder of the generator are comprised of standardized blocks of convolutional, batch normalization, dropout, and activation layers. This standardization means that we can develop helper functions to create each block of layers and call it repeatedly to build-up the encoder and decoder parts of the model.

### The define_generator() function below implements the U-Net encoder-decoder generator model. It uses the define_encoder_block() helper function to create blocks of layers for the encoder and the decoder_block() function to create blocks of layers for the decoder. The tanh activation function is used in the output layer, meaning that pixel values in the generated image will be in the range [-1,1].

In [ ]:
def encoding_block(inp, filters, bat=True):
    init = RandomNormal(stddev=0.02)
    d=Conv2D(filters, (4,4), strides=(2,2), padding='same', kernel_initializer=init)(inp)
    if bat:
        d=BatchNormalization()(d,training=True)
    d=LeakyReLU(alpha=0.2)(d)
    return d

def decoding_block(inp, skip_in,filters, dropout=True):
    init = RandomNormal(stddev=0.02)
    u=Conv2DTranspose(filters, (4,4), strides=(2,2), padding='same', kernel_initializer=init)(inp)
    u=BatchNormalization()(u, training=True)
    if dropout:
        u=Dropout(0.5)(u)
    u=Concatenate()([u, skip_in])
    u=Activation('relu')(u)
    return u
    

In [ ]:
def build_generator(image_shape):
    init = RandomNormal(stddev=0.02)
    inp_s=Input(shape=(256,256,3))
    
    d1=encoding_block(inp_s, 64, bat=False)
    d2=encoding_block(d1, 128)
    d3=encoding_block(d2, 256)
    d4=encoding_block(d3, 512)
    d5=encoding_block(d4, 512)
    d6=encoding_block(d5, 512)
    d7=encoding_block(d6, 512)
    
    bottleneck = Conv2D(512, (4,4), strides=(2,2), padding='same', kernel_initializer=init)(d7)
    bottleneck = Activation('relu')(bottleneck)
    
    u1 = decoding_block(bottleneck, d7, 512)
    u2 = decoding_block(u1, d6, 512)
    u3 = decoding_block(u2, d5, 512)
    u4 = decoding_block(u3, d4, 512, dropout=False)
    u5 = decoding_block(u4, d3, 256, dropout=False)
    u6 = decoding_block(u5, d2, 128, dropout=False)
    u7 = decoding_block(u6, d1, 64, dropout=False)

    last = Conv2DTranspose(3, (4,4), strides=(2,2), padding='same', kernel_initializer=init)(u7)
    out_image = Activation('tanh')(last)
    # define model
    model = Model(inp_s, out_image)
    return model

In [ ]:
image_shape=(256,256,3)
gen=build_generator(image_shape)
gen.summary()

In [ ]:
def build_gan(g_model, d_model, image_shape):
    
    for layer in d_model.layers:
        if not isinstance(layer, BatchNormalization):
            layer.trainable = False
            
    in_src = Input(shape=image_shape)
    gen_out = g_model(in_src)
    dis_out = d_model([in_src, gen_out])
    
    model = Model(in_src, [dis_out, gen_out])
    
    opt = Adam(lr=0.0002, beta_1=0.5)
    model.compile(loss=['binary_crossentropy', 'mae'], optimizer=opt, loss_weights=[1,100])
    return model

In [ ]:
build_gan(gen, disc, (256,256,3)).summary()

In [ ]:
def load_real_samples(train_s, train_t):
    train_s=(train_s-127.5)/127.5
    train_t=(train_t-127.5)/127.5
    
    return [train_s, train_t]

In [ ]:
def generate_real_samples(dataset, n_samples, patch_shape):
    train_s, train_t = dataset
    ix = np.random.randint(0, train_s.shape[0], n_samples)
    x1, x2 = train_s[ix], train_t[ix]
    y = np.ones((n_samples, patch_shape, patch_shape, 1))
    
    return [x1, x2], y

In [ ]:
def generate_fake_samples(generator, samples, patch_shape):
    fake = generator.predict(samples, verbose=0)
    y = np.zeros((len(fake), patch_shape, patch_shape, 1))
    return fake, y

In [ ]:
def summarize_performance(step, g_model, dataset, n_samples=3):
    # select a sample of input images
    [X_realA, X_realB], _ = generate_real_samples(dataset, n_samples, 1)
    # generate a batch of fake samples
    X_fakeB, _ = generate_fake_samples(g_model, X_realA, 1)
    # scale all pixels from [-1,1] to [0,1]
    X_realA = (X_realA + 1) / 2.0
    X_realB = (X_realB + 1) / 2.0
    X_fakeB = (X_fakeB + 1) / 2.0
    # plot real source images
    for i in range(n_samples):
        plt.subplot(3, n_samples, 1 + i)
        plt.axis('off')
        plt.imshow(X_realA[i])
    # plot generated target image
    for i in range(n_samples):
        plt.subplot(3, n_samples, 1 + n_samples + i)
        plt.axis('off')
        plt.imshow(X_fakeB[i])
    # plot real target image
    for i in range(n_samples):
        plt.subplot(3, n_samples, 1 + n_samples*2 + i)
        plt.axis('off')
        plt.imshow(X_realB[i])
    # save plot to file
    filename1 = 'plot_%06d.png' % (step+1)
    plt.savefig(filename1)
    plt.close()
    # save the generator model
    filename2 = 'model_%06d.keras' % (step+1)
    g_model.save(filename2)
    print('>Saved: %s and %s' % (filename1, filename2))

In [ ]:
from tqdm import tqdm
def train(d_model, g_model, gan_model, dataset, n_epochs=100, n_batch=1):

    n_patch = d_model.output_shape[1]

    trainA, trainB = dataset

    bat_per_epo = int(len(trainA) / n_batch)

    n_steps = bat_per_epo * n_epochs

    for i in tqdm(range(n_steps)):
        
        [X_realA, X_realB], y_real = generate_real_samples(dataset, n_batch, n_patch)
        X_fakeB, y_fake = generate_fake_samples(g_model, X_realA, n_patch)

        d_loss1 = d_model.train_on_batch([X_realA, X_realB], y_real)
        d_loss2 = d_model.train_on_batch([X_realA, X_fakeB], y_fake)

        g_loss, _, _ = gan_model.train_on_batch(X_realA, [y_real, X_realB])

        #print('>%d, d1[%.3f] d2[%.3f] g[%.3f]' % (i+1, d_loss1, d_loss2, g_loss))

        if (i+1) % (bat_per_epo * 10) == 0:
            summarize_performance(i, g_model, dataset)

In [ ]:
dataset = load_real_samples(train_s, train_t)
print('Loaded', dataset[0].shape, dataset[1].shape)

image_shape = dataset[0].shape[1:]

d_model = build_discriminator(image_shape)
g_model = build_generator(image_shape)

gan_model = build_gan(g_model, d_model, image_shape)

train(d_model, g_model, gan_model, dataset, n_epochs=200)

In [ ]:
generator=keras.models.load_model('/kaggle/input/modell/model_054800.keras')


In [ ]:
train_s

In [ ]:
train_s, train_t = load_real_samples(train_s, train_t)
val_s, val_t = load_real_samples(val_s, val_t)

In [ ]:
ix=np.random.randint(0,1096-3,1)[0]
imgs,_ = generate_fake_samples(generator, val_s[ix:ix+3], 16)

for i in range(3):
    plt.subplot(3,3,i+1)
    plt.imshow(val_s[ix:ix+3][i])
    
for i in range(3,6):
    plt.subplot(3,3 ,i+1)
    plt.imshow(val_t[ix:ix+3][i-3])
    
for i in range(6,9):
    plt.subplot(3,3 ,i+1)
    plt.imshow(imgs[i-6])
    
plt.show()

In [ ]:
from PIL import Image

i=plt.imread('/kaggle/input/images/Screenshot 2023-12-05 at 6.38.18 PM.png')
i=i[:,:,:3]

original_image = Image.open("/kaggle/input/images/Screenshot 2023-12-05 at 6.38.18 PM.png")

# Define the new size (width, height)
new_size = (256, 256)

# Resize the image
resized_image = original_image.resize(new_size)
img=np.array(resized_image)[:,:,:3]
img=(img-127.5)/127.5


In [ ]:
plt.imshow(img)

In [ ]:
img,_=generate_fake_samples(generator, np.expand_dims(img, axis=0), 16)
plt.imshow(img[0])
plt.show()

In [ ]:
plt.imshow(train_s[52])
plt.show()